In [1]:
import numpy as np
import pandas as pd
# df=pd.read_excel("allfeatures.xlsx")
df=pd.read_csv("allfeatures.csv")
print(f'shapte of feateures: {df.shape}')
print(f'total na values: {df.isna().sum().sum()}')
print(f'total null values: {df.isnull().sum().sum()}')

shapte of feateures: (7722, 1389)
total na values: 0
total null values: 0


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

def tune_hyperparameters(X, y):
    # Define the XGBoost classifier model
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        n_jobs=-1  # Utilize all available CPUs for xgboost
    )

    # Define the parameter grid to search
    params = {
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'n_estimators': [100, 200, 300, 400],
        'colsample_bytree': [0.5, 0.7, 0.9, 1],
        'subsample': [0.5, 0.7, 0.9, 1],
        'gamma': [0, 0.1, 0.2, 0.3]
    }

    # Setup GridSearchCV to find the best hyperparameters
    # n_jobs=-1 uses all available CPUs for the grid search
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=params,
        scoring='roc_auc',
        cv=5,
        verbose=1,
        n_jobs=-1  # Utilize all available CPUs for GridSearchCV
    )

    # Fit GridSearchCV on the provided training data
    grid_search.fit(X, y)

    # Output the best parameters and the best AUC score achieved
    print("Best parameters:", grid_search.best_params_)
    print("Best AUC: {:.4f}".format(grid_search.best_score_))

    # Return the model with the best parameters
    return grid_search.best_estimator_

# Feature matrix and label vector as specified
X = df.drop(['pcid', 'class', 'smile'], axis=1)
y = df['class']

# Call the function with your dataset
best_model = tune_hyperparameters(X, y)

In [4]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler

# Assuming 'df' is your DataFrame with the necessary columns
X = df.drop(['class', 'pcid', 'smile'], axis=1)  # feature matrix
y = df['class']  # label vector (binary)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling for better performance of XGBoost not necessary but can be performed if part of your preprocessing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the XGBoost classifier
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the model
model.fit(X_train_scaled, y_train)

# Predicting the test set results
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]  # probabilities for positive class

# Calculating the evaluation metrics
auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# Printing the metrics
print(f"ROC AUC Score: {auc:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Accuracy: {accuracy:.2f}")

ROC AUC Score: 0.94
F1 Score: 0.86
Precision: 0.86
Recall: 0.86
Accuracy: 0.87


In [6]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler

# Assuming 'df' is your DataFrame and it's already loaded
X = df.drop(['class', 'smile', 'pcid'], axis=1)  # feature matrix
y = df['class']  # label vector (binary)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling for XGBoost which is not necessary but included for completeness
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Create the parameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

# Setup the grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Retrieve the best model from the grid search
best_model = grid_search.best_estimator_

# Predicting the test set results
y_pred = best_model.predict(X_test_scaled)
y_proba = best_model.predict_proba(X_test_scaled)[:, 1]

# Calculating the evaluation metrics
auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# Printing the metrics
print(f"ROC AUC Score: {auc:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Accuracy: {accuracy:.2f}")

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 200}
Best cross-validation score: 0.87
ROC AUC Score: 0.95
F1 Score: 0.87
Precision: 0.87
Recall: 0.87
Accuracy: 0.88


In [26]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [4, 5, 6, 7, 8]
        }

xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic', nthread=2)

folds = 10
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=10, cv=skf.split(X_train_scaled, y_train), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train_scaled, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 5 candidates, totalling 50 fits

 Time taken: 0 hours 2 minutes and 49.78 seconds.


In [27]:
#print('\n All results:')
#print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)


 Best estimator:
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=600, n_jobs=None, nthread=2,
              num_parallel_tree=None, ...)

 Best normalized gini score for 10-fold search with 5 parameter combinations:
0.8929648651599802

 Best hyperparameters:
{'subsample': 0.8, 'min_child_weight': 5, 'max_depth': 7, 'gamma': 1, 'colsample_bytree': 1.0}


In [28]:
y_pred = random_search.predict(X_test_scaled)
y_proba = random_search.predict_proba(X_test_scaled)[:, 1]
 
# Calculating the evaluation metrics
auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
 
# Printing the metrics 
print(f"ROC AUC Score: {auc:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Accuracy: {accuracy:.2f}")

ROC AUC Score: 0.95
F1 Score: 0.88
Precision: 0.88
Recall: 0.87
Accuracy: 0.88
